In [20]:
import pandas as pd 
import numpy as np 
import math 
import time

Yunus Emre Uluer 2210356102

In [21]:
f = pd.read_csv('16P.csv')
f.drop(columns='Response Id')
personalitydict = {'ESTJ' : 0 , 'ENTJ' : 1, 'ESFJ' : 2 ,'ENFJ' : 3 ,'ISTJ' : 4 ,'ISFJ' : 5,'INTJ' : 6,'INFJ' : 7 ,'ESTP' : 8	,'ESFP' : 9,'ENTP' : 10 ,'ENFP' : 11 ,'ISTP' : 12,'ISFP' : 13	,'INTP' : 14,'INFP' : 15}
f.drop(columns='Response Id' , inplace=True)
f.replace(personalitydict, inplace=True)


The basic part of the code

In [27]:
def KNN(k,train,test,trained_personality,actual_personality) : 

    euclidian_distance = np.sqrt(np.sum(train**2, axis=1) + np.sum(test**2, axis=1)[:, np.newaxis]+ -2 * np.dot(test, train.T)) 
    #euclidian distance it uses matrix multiplicaton. The shape of euclidian distance is (48000,12000) this means for every row 
    # is an array which gives the distance between trained value. euclidian distance is (x-y)square. This line uses xsquare + ysquare-2xy

     
    closest_k = np.argpartition(euclidian_distance, k, axis=1)[:, :k] # It gives the indexes of closest k's .And this indexes will be the first k value.
    #After it sorts(kind of) it takes those values to an array and proceed.
    closest_k_personalities = trained_personality[closest_k] #trained personality has the same indexes as euclidian distance so this indexes gives personality encodes.

    predicted_personality = []
    for i in closest_k_personalities:
        predicted_personality.append(np.bincount(i).argmax())     #We get the max repeated value and predict it. The array is 1dimensional now
    data = {'Actual' : actual_personality , 'Predicted' : predicted_personality}
    datas =   pd.DataFrame(data)

    confusion_matrix = pd.crosstab(datas['Actual'], datas['Predicted'], )
    confusion_matrix = confusion_matrix.to_numpy()


    TP_FN = confusion_matrix.sum(1)    #axis 1 sum gives TP_FN
    TP_FP = confusion_matrix.sum(0)    #axis 0 sum gives TP_FP
    TP = confusion_matrix.diagonal()   #diagonal one gives TP only

    FP_FN = (TP_FN + TP_FP) -2*TP    #math calculations for FP_FN

    precision = ((TP / TP_FP).sum())/16 #macro
    recall = ((TP / TP_FN).sum())/16
    accuracy =(((confusion_matrix.sum()-(FP_FN)) /confusion_matrix.sum()).sum())/16
    return precision,recall,accuracy


Fold Function : 


In [28]:
def fold_for_k(j):
    for k in [1,2,3,4,5] : 
        k = k-1
        Y = f['Personality']
        Y = Y.to_numpy()
        f.drop(columns='Personality')
        array = f.to_numpy()
        normalized_array = (array-3)/6  #normalization

        train = np.delete(array,slice(12000*k,12000*(k+1)-1),0)      #splitting the data
        test = array[12000*k:12000*(k+1)-1]

        normalizedtrain = np.delete(normalized_array, (slice(12000*k,12000*(k+1)-1)),0)
        normalizedtest = normalized_array[12000*k:12000*(k+1)-1]

        trainedpersonality= np.delete(Y, slice(12000*k,12000*(k+1)-1))
        actualpersonality = Y[12000*k:12000*(k+1)-1]   

        withoutnormalization   =  KNN(j,train,test,trainedpersonality,actualpersonality)
        withoutnormalization_100 =  tuple(100 * elem for elem in withoutnormalization)    #getting the values
        normalization = KNN(j,normalizedtrain,normalizedtest,trainedpersonality,actualpersonality)
        normalization_100 =  tuple(100 * elem for elem in normalization)
        
        
        df = { f'K-NN for {j}, Fold-{k+1}': ['Without Normalization','Normalization'],'Accuracy': [withoutnormalization_100[2],normalization_100[2]],'Recall': [withoutnormalization_100[1],normalization_100[1]], 'Precision': [withoutnormalization_100[0],normalization_100[0]] }
        df = pd.DataFrame(df)    
        df.set_index(f'K-NN for {j}, Fold-{k+1}',inplace=True)
        print(df,'\n')



In [29]:
fold_for_k(1)

                        Accuracy     Recall  Precision
K-NN for 1, Fold-1                                    
Without Normalization  99.873948  98.989658  98.994213
Normalization          99.872906  98.981774  98.986069 

                        Accuracy     Recall  Precision
K-NN for 1, Fold-2                                    
Without Normalization  99.867697  98.941353  98.941354
Normalization          99.868739  98.949300  98.949499 

                        Accuracy     Recall  Precision
K-NN for 1, Fold-3                                    
Without Normalization  99.876031  99.011218  99.019163
Normalization          99.879157  99.035874  99.043223 

                        Accuracy     Recall  Precision
K-NN for 1, Fold-4                                    
Without Normalization  99.868739  98.951303  98.947875
Normalization          99.868739  98.951303  98.947843 

                        Accuracy     Recall  Precision
K-NN for 1, Fold-5                                    
Wi

In [30]:
fold_for_k(3)

                        Accuracy     Recall  Precision
K-NN for 3, Fold-1                                    
Without Normalization  99.890616  99.122937  99.127080
Normalization          99.890616  99.122937  99.126969 

                        Accuracy     Recall  Precision
K-NN for 3, Fold-2                                    
Without Normalization  99.879157  99.032504  99.032678
Normalization          99.879157  99.032504  99.033222 

                        Accuracy     Recall  Precision
K-NN for 3, Fold-3                                    
Without Normalization  99.893741  99.153871  99.154064
Normalization          99.893741  99.153828  99.154267 

                        Accuracy     Recall  Precision
K-NN for 3, Fold-4                                    
Without Normalization  99.880198  99.041533  99.037756
Normalization          99.880198  99.041533  99.038064 

                        Accuracy     Recall  Precision
K-NN for 3, Fold-5                                    
Wi

In [31]:
fold_for_k(5)

                        Accuracy     Recall  Precision
K-NN for 5, Fold-1                                    
Without Normalization  99.893741  99.147919  99.152460
Normalization          99.893741  99.147919  99.152553 

                        Accuracy     Recall  Precision
K-NN for 5, Fold-2                                    
Without Normalization  99.885407  99.083354  99.084556
Normalization          99.885407  99.083354  99.084342 

                        Accuracy     Recall  Precision
K-NN for 5, Fold-3                                    
Without Normalization  99.895825  99.169418  99.174332
Normalization          99.895825  99.169418  99.175292 

                       Accuracy    Recall  Precision
K-NN for 5, Fold-4                                  
Without Normalization  99.88124  99.04942  99.045091
Normalization          99.88124  99.04942  99.044994 

                        Accuracy     Recall  Precision
K-NN for 5, Fold-5                                    
Without No

In [32]:
fold_for_k(7)

                        Accuracy     Recall  Precision
K-NN for 7, Fold-1                                    
Without Normalization  99.893741  99.147937  99.150723
Normalization          99.893741  99.147937  99.150631 

                        Accuracy     Recall  Precision
K-NN for 7, Fold-2                                    
Without Normalization  99.882282  99.058302  99.059128
Normalization          99.881240  99.050330  99.049521 

                        Accuracy     Recall  Precision
K-NN for 7, Fold-3                                    
Without Normalization  99.894783  99.161414  99.167028
Normalization          99.894783  99.161414  99.166462 

                        Accuracy     Recall  Precision
K-NN for 7, Fold-4                                    
Without Normalization  99.880198  99.040336  99.036229
Normalization          99.880198  99.040336  99.036592 

                        Accuracy     Recall  Precision
K-NN for 7, Fold-5                                    
Wi

In [33]:
fold_for_k(9)

                        Accuracy     Recall  Precision
K-NN for 9, Fold-1                                    
Without Normalization  99.894783  99.156128  99.158824
Normalization          99.894783  99.156128  99.158732 

                        Accuracy     Recall  Precision
K-NN for 9, Fold-2                                    
Without Normalization  99.883324  99.067453  99.065443
Normalization          99.884365  99.075344  99.073641 

                        Accuracy     Recall  Precision
K-NN for 9, Fold-3                                    
Without Normalization  99.894783  99.160758  99.166219
Normalization          99.894783  99.160758  99.166227 

                        Accuracy     Recall  Precision
K-NN for 9, Fold-4                                    
Without Normalization  99.879157  99.032127  99.029579
Normalization          99.880198  99.039703  99.037333 

                        Accuracy     Recall  Precision
K-NN for 9, Fold-5                                    
Wi

Every fold value in 5-fold cross validation have similar values on every k. This means the algorithm does not rely on  positions of the data and can be trusted(with 5-fold cross validation). 

Normalization did not effect the accuracy, recall,precision. Because normalization effects when the classes are not scaled. In our data every class(whis is a question) takes values between -3 and 3 so it is already scaled. The diffrence between normalization and without-normalization should be because of the errors due to float values.    

The best K-NN value which gives the most accuracy is 5 . But it is so similar when K equals 9. The best value of K can be calculated by test so The value 5-9 can be used. Also 3-7 is accurate. The number of test can be increased for better examination. Recall and Precision is nearly the same as accuracy. The best ones are 5-9. But the diffrence is very low.

Errors : Because every class are not cumulated in the same place, when euclidian_distance values are refers diffrent personalities algorithm chooses it randomly. This cause errors. 